In [1]:
"""
clone and install PySDM dependencies in Colab
(to use GPU on Colab set hardware accelerator to 'GPU' before session start
in the "Runtime :: Change runtime type :: Hardware accelerator" menu)
"""
import os, sys
if 'google.colab' in sys.modules:
    %cd /content
    if not os.path.isdir('PySDM'):
        !git clone --depth 1 https://github.com/atmos-cloud-sim-uj/PySDM.git
    %cd PySDM
    !pip install -r requirements.txt

In [ ]:
import os, sys
if 'google.colab' in sys.modules:
    %cd /content/PySDM
else:
    sys.path.insert(0, os.path.join(os.getcwd(), '../..'))

In [ ]:
from numpy import errstate
from PySDM.backends import Default, ThrustRTC
from PySDM.dynamics.coalescence.kernels import Geometric, Hydrodynamic, Electric
from PySDM_examples.Berry_1967_Figs.spectrum_plotter import SpectrumPlotter
from PySDM_examples.Berry_1967_Figs.setup import Setup
from PySDM_examples.Shima_et_al_2009_Fig_2.example import run
import ipywidgets as widgets

In [3]:
progbar = widgets.IntProgress(min=0, max=100, description='%')
class ProgbarUpdater:
    def __init__(self, progbar, max_steps):
        self.max_steps = max_steps
        self.steps = 0
        self.progbar = progbar

    def notify(self):
        self.steps += 1
        self.progbar.value = 100 * (self.steps / self.max_steps)
        
class WidgetsFreezer:
    def __init__(self, widgets):
        self.widgets = widgets
        
    def observe(*args, **kwargs):
        pass
    
    @property
    def value(self):
        return self
    
    def __enter__(self):
        for widget in self.widgets:
            widget.disabled = True
        return self
            
    def __exit__(self, *args, **kwargs):
        for widget in self.widgets:
            widget.disabled = False

In [4]:
def demo(*, freezer, n_SD, n_step, n_plot, kernel, adaptive, smooth, gpu):
    with freezer:
        with errstate(all='raise'):
            setup = Setup()
            setup.backend = ThrustRTC() if gpu else Default()
            setup.n_sd = 2 ** n_SD
            setup.adaptive = adaptive
            setup.dt = 10 if adaptive else setup.dt
            setup._steps = [i * (n_step // n_plot) for i in range(n_plot + 1)]
            if kernel == 'geometric sweep-out':
                setup.kernel = Geometric()
            elif kernel == 'electric field 3000V/cm':
                setup.kernel = Electric()
            else:
                setup.kernel = Hydrodynamic()

            states, _ = run(setup, (ProgbarUpdater(progbar, setup.steps[-1]),))

        with errstate(invalid='ignore'):
            plotter = SpectrumPlotter(setup)
            plotter.smooth = smooth
            for step, state in states.items():
                plotter.plot(state, step * setup.dt)
            plotter.show()

In [5]:
n_SD = widgets.IntSlider(value=14, min=12, max=18, step=1, description='$log_2(n_{SD})$', continuous_update=False)
n_step = widgets.IntSlider(value=1000, step=100, min=0, max=2400, description='$n_{step}$', continuous_update=False)
n_plot = widgets.IntSlider(value=10, step=1, min=1, max=16, description='$n_{plot}$', continuous_update=False)
sliders = widgets.HBox([n_SD, n_step, n_plot])

adaptive = widgets.Checkbox(value=False, description='adaptive dt')
smooth = widgets.Checkbox(value=True, description='smooth plot')
gpu = widgets.Checkbox(value=False, description='GPU')
options = [adaptive, smooth]
if ThrustRTC.ENABLE:
    options.append(gpu)
kernel = widgets.Select(
    options=['geometric sweep-out', 'electric field 3000V/cm', 'hydrodynamic capture'],
    value='geometric sweep-out',
    description='kernel:',
    rows=3
)
options.append(kernel)
boxes = widgets.HBox(options)
    
freezer = WidgetsFreezer([n_SD, n_step, n_plot, kernel, adaptive, gpu])
self = widgets.interactive_output(demo, 
                                  {'freezer': freezer, 'n_SD': n_SD, 'n_step': n_step, 'n_plot': n_plot, 
                                   'kernel': kernel, 'adaptive': adaptive, 'smooth': smooth, 'gpu': gpu})

display(sliders, boxes, progbar, self)

IntProgress(value=100, description='%')

Output()